# Imports

In [1]:
# pip install autogluon

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

# Load Data

In [3]:
df = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_1999_2023.pkl")

## DF Columns By Category

| Category                     | Columns                                  |
|------------------------------|------------------------------------------|
| **Passing Statistics**       | completions, attempts, passing_yards, passing_tds, interceptions, sacks, sack_yards, sack_fumbles, sack_fumbles_lost, passing_air_yards, passing_yards_after_catch, passing_first_downs, passing_epa, passing_2pt_conversions, pacr, dakota |
| **Rushing Statistics**       | carries, rushing_yards, rushing_tds, rushing_fumbles, rushing_fumbles_lost, rushing_first_downs, rushing_epa, rushing_2pt_conversions, ry_sh, rtd_sh, rfd_sh, rtdfd_sh |
| **Receiving Statistics**     | receptions, targets, receiving_yards, receiving_tds, receiving_fumbles, receiving_fumbles_lost, receiving_air_yards, receiving_yards_after_catch, receiving_first_downs, receiving_epa, receiving_2pt_conversions, racr, target_share, air_yards_share, wopr_x |
| **Fantasy and Special Teams**| special_teams_tds, fantasy_points, fantasy_points_ppr |
| **General Game Statistics**  | games                                    |
| **Player Information**       | player_id, season, season_type, position, player_name, age, team, rank, tier, Draft Year, Draft No., Draft Round, Draft Pick, Draft Overall, Draft Team, College |
| **Passing Efficiency Metrics**| avg_time_to_throw, avg_completed_air_yards, avg_intended_air_yards, avg_air_yards_differential, aggressiveness, max_completed_air_distance, avg_air_yards_to_sticks, pass_yards, pass_touchdowns, passer_rating, completion_percentage, expected_completion_percentage, completion_percentage_above_expectation, avg_air_distance, max_air_distance, efficiency, percent_attempts_gte_eight_defenders, avg_time_to_los |
| **Rushing Efficiency Metrics**| rush_attempts, rush_yards, expected_rush_yards, rush_yards_over_expected, avg_rush_yards, rush_yards_over_expected_per_att, rush_pct_over_expected, rush_touchdowns |
| **Receiving Efficiency Metrics**| avg_cushion, avg_separation, avg_intended_air_yards_receiving, percent_share_of_intended_air_yards, catch_percentage, yards, rec_touchdowns, avg_yac, avg_expected_yac, avg_yac_above_expectation |


# Filter By Position

In [4]:
df_wr = df.copy().query("position == 'WR'")
print(f"Length of wide receiver data set: {len(df_wr)}")

Length of wide receiver data set: 5039


In [5]:
# remove our na's
df_wr.dropna(subset="Draft Year", inplace=True)
print(f"After removing NA draft rows, data set size: {len(df_wr)}")

After removing NA draft rows, data set size: 4809


In [6]:
df_wr['Draft_Year'] = df_wr['Draft Year'].astype(str).str.replace('s', '')
df_wr["Draft_Year"] = df_wr["Draft_Year"].astype(int)

df_wr["Draft_Round"] = df_wr["Draft Round"].astype(str).str.replace('Undrafted', '8')
df_wr["Draft_Round"] = df_wr["Draft_Round"].astype(int)

df_wr["Draft_Overall"] = df_wr["Draft Overall"].astype(str).str.replace('Undrafted', '400')
df_wr["Draft_Overall"] = df_wr["Draft_Overall"].astype(int)

# WR Relveant Columns

In [7]:
wr_cols = [
    'player_id', 'season',         
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
    'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
    'wopr_x', 'fantasy_points', 'fantasy_points_ppr',
    'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y', 'ry_sh', 'rtd_sh',
    'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa', 'ppr_sh', 
    'position', 'player_name', 'age', 'team', 'rank', 'tier', 
    'Draft_Year', 'Draft_Round', 'Draft_Overall',
    'Draft Team', 'College', 'avg_cushion', 'avg_separation',
    'avg_intended_air_yards_receiving', 'percent_share_of_intended_air_yards', 
    'catch_percentage', 'yards', 'rec_touchdowns', 'avg_yac', 'avg_expected_yac',
    'avg_yac_above_expectation', 'pfr_player_id', 'offense_snaps', 'offense_pct', 
    'gs_pfr_rec', 'tgt_pfr_rec', 'rec_pfr_rec', 'yds_pfr_rec', 'td_pfr_rec',
    'x1d_pfr_rec', 'ybc_pfr_rec', 'ybc_r_pfr_rec', 'yac_pfr_rec',
    'yac_r_pfr_rec', 'adot_pfr_rec', 'brk_tkl_pfr_rec', 'rec_br_pfr_rec',
    'drop_pfr_rec', 'drop_percent_pfr_rec', 'int_pfr_rec', 'rat_pfr_rec'
]

In [8]:
df_wr = df_wr[wr_cols]

In [9]:
df_wr.columns

Index(['player_id', 'season', 'receptions', 'targets', 'receiving_yards',
       'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_air_yards', 'receiving_yards_after_catch',
       'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',
       'racr', 'target_share', 'air_yards_share', 'wopr_x', 'fantasy_points',
       'fantasy_points_ppr', 'games', 'tgt_sh', 'ay_sh', 'yac_sh', 'wopr_y',
       'ry_sh', 'rtd_sh', 'rfd_sh', 'rtdfd_sh', 'dom', 'w8dom', 'yptmpa',
       'ppr_sh', 'position', 'player_name', 'age', 'team', 'rank', 'tier',
       'Draft_Year', 'Draft_Round', 'Draft_Overall', 'Draft Team', 'College',
       'avg_cushion', 'avg_separation', 'avg_intended_air_yards_receiving',
       'percent_share_of_intended_air_yards', 'catch_percentage', 'yards',
       'rec_touchdowns', 'avg_yac', 'avg_expected_yac',
       'avg_yac_above_expectation', 'pfr_player_id', 'offense_snaps',
       'offense_pct', 'gs_pfr_rec', 'tgt_pfr_rec', 'rec_p

In [10]:
df_wr["season"].dtype

dtype('int32')

# Get Lag Version

For having the previous season as features and fantasy points/receiving first downs be the labels.

In [11]:
def create_lag_df(df, cols_to_filter=3, col_to_increment="season", 
                  cols_to_merge=["player_id", "season"]
    ):
    """"""
    df_now = df.copy()
    df_last = df.copy()

    rename_dict = {}
    for col in list(df_last.columns[cols_to_filter:]):
        rename_dict[col] = f"{col}_last"

    df_last.rename(columns=rename_dict, inplace=True)
    df_last[col_to_increment] += 1

    df_lag = df_now.merge(df_last, how='inner', on=cols_to_merge)

    return df_lag

In [12]:
df_lag = create_lag_df(df_wr.copy(), cols_to_filter=2)

In [13]:
df_lag.query("player_name == 'Mike Evans' and season == 2017")[["player_name", "season", 
                                                                "receiving_yards", "receiving_yards_last"]]

,player_name,season,receiving_yards,receiving_yards_last
2664,Mike Evans,2017,1001.0,1321.0


# Visualize Dataset

In [14]:
df_vis = df_lag \
    .groupby(by="season") \
    .agg({"fantasy_points_ppr": ["mean", "max", "min", "median", "sum"]})
df_vis.columns = ['_'.join(col).strip() for col in df_vis.columns.values]
df_vis.reset_index(inplace=True)

In [15]:
fig = px.line(df_vis, x="season", y=["fantasy_points_ppr_mean", "fantasy_points_ppr_max", 
                                     "fantasy_points_ppr_min", "fantasy_points_ppr_median",
                                     "fantasy_points_ppr_sum"
                                    ])
fig.write_html("../../interactive-2.0/appendix/fantasy_point_ppr_agg_by_season.html")
fig.show()

In [16]:
df_vis_2 = df_lag.copy()
df_vis_2["Tier"] = df_vis_2['tier'].astype(str)

In [17]:
cat_order = ['1.0',
 '2.0',
 '3.0',
 '4.0',
 '5.0',
 '6.0',
 '7.0',
 '8.0',
 '9.0',
 '10.0',
 '11.0',
 '12.0',
 '13.0',
 '14.0',
 '15.0',
 '16.0',
 '17.0',
 '18.0',
 '19.0',
 '20.0',
]

In [18]:
fig = px.scatter(df_vis_2, x="season", y=["fantasy_points_ppr"], color="Tier",
                 category_orders={"Tier": cat_order}
                )

# Update layout for clarity.
fig.update_layout(xaxis_title='Season', yaxis_title='Fantasy Points PPR')
fig.write_html("../../interactive-2.0/appendix/fantasy_point_ppr_by_season.html")
fig.show()

# Select Feature Columns

I.e. the columns with last in them, and of numeric type.

In [19]:
non_numeric_cols = df_lag.select_dtypes(exclude=[np.number]).columns

In [20]:
non_numeric_cols

Index(['player_id', 'position', 'player_name', 'team', 'Draft Team', 'College',
       'pfr_player_id', 'position_last', 'player_name_last', 'team_last',
       'Draft Team_last', 'College_last', 'pfr_player_id_last'],
      dtype='object')

In [21]:
assert(df_lag["Draft_Year"].dtype == 'int64')
assert(df_lag["Draft_Round"].dtype == 'int64')
assert(df_lag["Draft_Overall"].dtype == 'int64')

In [22]:
[col for col in df_lag.columns if '_last' in col]

['receptions_last',
 'targets_last',
 'receiving_yards_last',
 'receiving_tds_last',
 'receiving_fumbles_last',
 'receiving_fumbles_lost_last',
 'receiving_air_yards_last',
 'receiving_yards_after_catch_last',
 'receiving_first_downs_last',
 'receiving_epa_last',
 'receiving_2pt_conversions_last',
 'racr_last',
 'target_share_last',
 'air_yards_share_last',
 'wopr_x_last',
 'fantasy_points_last',
 'fantasy_points_ppr_last',
 'games_last',
 'tgt_sh_last',
 'ay_sh_last',
 'yac_sh_last',
 'wopr_y_last',
 'ry_sh_last',
 'rtd_sh_last',
 'rfd_sh_last',
 'rtdfd_sh_last',
 'dom_last',
 'w8dom_last',
 'yptmpa_last',
 'ppr_sh_last',
 'position_last',
 'player_name_last',
 'age_last',
 'team_last',
 'rank_last',
 'tier_last',
 'Draft_Year_last',
 'Draft_Round_last',
 'Draft_Overall_last',
 'Draft Team_last',
 'College_last',
 'avg_cushion_last',
 'avg_separation_last',
 'avg_intended_air_yards_receiving_last',
 'percent_share_of_intended_air_yards_last',
 'catch_percentage_last',
 'yards_last',
 

In [23]:
feature_columns = [
    'receptions_last',
    'targets_last',
    'receiving_yards_last',
    'receiving_tds_last',
    'receiving_fumbles_last',
    'receiving_fumbles_lost_last',
    'receiving_air_yards_last',
    'receiving_yards_after_catch_last',
    'receiving_first_downs_last',
    'receiving_epa_last',
    'receiving_2pt_conversions_last',
    'racr_last',
    'target_share_last',
    'air_yards_share_last',
    'wopr_x_last',
    # 'fantasy_points_last',
    # 'fantasy_points_ppr_last',
    'games_last',
    'tgt_sh_last',
    'ay_sh_last',
    'yac_sh_last',
    'wopr_y_last',
    'ry_sh_last',
    'rtd_sh_last',
    'rfd_sh_last',
    'rtdfd_sh_last',
    'dom_last',
    'w8dom_last',
    'yptmpa_last',
    'ppr_sh_last',
    'age_last',
    'rank_last',
    'tier_last',
    'Draft_Year_last',
    'Draft_Round_last',
    'Draft_Overall_last',
    'avg_cushion_last',
    'avg_separation_last',
    'avg_intended_air_yards_receiving_last',
    'percent_share_of_intended_air_yards_last',
    'catch_percentage_last',
#     'yards_last',  # same as receiving yards - duplicate column
    'rec_touchdowns_last',
    'avg_yac_last',
    'avg_expected_yac_last',
    'avg_yac_above_expectation_last'
]

In [24]:
label = "fantasy_points_ppr_last"

In [25]:
cols_to_norm = feature_columns + [label]
df_to_norm = df_lag[cols_to_norm].copy()

In [26]:
cols_to_norm

['receptions_last',
 'targets_last',
 'receiving_yards_last',
 'receiving_tds_last',
 'receiving_fumbles_last',
 'receiving_fumbles_lost_last',
 'receiving_air_yards_last',
 'receiving_yards_after_catch_last',
 'receiving_first_downs_last',
 'receiving_epa_last',
 'receiving_2pt_conversions_last',
 'racr_last',
 'target_share_last',
 'air_yards_share_last',
 'wopr_x_last',
 'games_last',
 'tgt_sh_last',
 'ay_sh_last',
 'yac_sh_last',
 'wopr_y_last',
 'ry_sh_last',
 'rtd_sh_last',
 'rfd_sh_last',
 'rtdfd_sh_last',
 'dom_last',
 'w8dom_last',
 'yptmpa_last',
 'ppr_sh_last',
 'age_last',
 'rank_last',
 'tier_last',
 'Draft_Year_last',
 'Draft_Round_last',
 'Draft_Overall_last',
 'avg_cushion_last',
 'avg_separation_last',
 'avg_intended_air_yards_receiving_last',
 'percent_share_of_intended_air_yards_last',
 'catch_percentage_last',
 'rec_touchdowns_last',
 'avg_yac_last',
 'avg_expected_yac_last',
 'avg_yac_above_expectation_last',
 'fantasy_points_ppr_last']

# Normalize the Dataset

In [27]:
def min_max_scaling(df, cols_to_norm):
    for col in cols_to_norm:
        max_v = df[col].max()
        min_v = df[col].min()
        
        df[f"{col}_norm"] = (df[col] - min_v) / (max_v - min_v)
        
    return df

In [28]:
df_norm = min_max_scaling(df_to_norm.copy(), cols_to_norm)

In [29]:
df_norm.head()

,receptions_last,targets_last,receiving_yards_last,receiving_tds_last,receiving_fumbles_last,receiving_fumbles_lost_last,receiving_air_yards_last,receiving_yards_after_catch_last,receiving_first_downs_last,receiving_epa_last,...,avg_cushion_last_norm,avg_separation_last_norm,avg_intended_air_yards_receiving_last_norm,percent_share_of_intended_air_yards_last_norm,catch_percentage_last_norm,rec_touchdowns_last_norm,avg_yac_last_norm,avg_expected_yac_last_norm,avg_yac_above_expectation_last_norm,fantasy_points_ppr_last_norm
0,30,60,296.0,1,1.0,0.0,0.0,0.0,18.0,-4.798412,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.155060
1,15,33,232.0,4,0.0,0.0,0.0,0.0,13.0,9.822120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.146921
2,3,6,26.0,0,0.0,0.0,0.0,0.0,2.0,-3.555738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018947
3,16,28,202.0,0,1.0,1.0,0.0,0.0,10.0,-6.491849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097178
4,3,5,23.0,0,0.0,0.0,0.0,0.0,2.0,0.845925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018269


In [30]:
df_ML = df_norm[df_norm.columns[len(cols_to_norm):]].copy()

In [31]:
df_ML.head()

,receptions_last_norm,targets_last_norm,receiving_yards_last_norm,receiving_tds_last_norm,receiving_fumbles_last_norm,receiving_fumbles_lost_last_norm,receiving_air_yards_last_norm,receiving_yards_after_catch_last_norm,receiving_first_downs_last_norm,receiving_epa_last_norm,...,avg_cushion_last_norm,avg_separation_last_norm,avg_intended_air_yards_receiving_last_norm,percent_share_of_intended_air_yards_last_norm,catch_percentage_last_norm,rec_touchdowns_last_norm,avg_yac_last_norm,avg_expected_yac_last_norm,avg_yac_above_expectation_last_norm,fantasy_points_ppr_last_norm
0,0.201342,0.292683,0.152008,0.043478,0.2,0.00,0.011845,0.003534,0.193548,0.233803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.155060
1,0.100671,0.160976,0.119471,0.173913,0.0,0.00,0.011845,0.003534,0.139785,0.304575,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.146921
2,0.020134,0.029268,0.014743,0.000000,0.0,0.00,0.011845,0.003534,0.021505,0.239818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018947
3,0.107383,0.136585,0.104220,0.000000,0.2,0.25,0.011845,0.003534,0.107527,0.225605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.097178
4,0.020134,0.024390,0.013218,0.000000,0.0,0.00,0.011845,0.003534,0.021505,0.261125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018269


# Create Train, Test, Validation Splits

In [32]:
X = df_ML.iloc[:, :-1]  # features
y = df_ML.iloc[:, -1]   # label

In [33]:
# First, split into train and temporary sets (train + validation, test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Now, split the train_temp into actual train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_temp, y_train_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# AutoGluon

In [34]:
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [35]:
label_column = 'fantasy_points_ppr_last_norm' 

## Fit

In [36]:
predictor = TabularPredictor(label=label_column).fit(
    train_data=train_data,
    tuning_data=val_data  # Optional, only if you want to use a separate validation set
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240510_000108"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240510_000108"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.9.7
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 23.2.0: Wed Nov 15 21:53:34 PST 

## Evaluation

In [37]:
performance = predictor.evaluate(test_data)
print(performance)

{'root_mean_squared_error': -0.00913531099541493, 'mean_squared_error': -8.345390698294891e-05, 'mean_absolute_error': -0.006215231695453841, 'r2': 0.9977234521027533, 'pearsonr': 0.9988676995213103, 'median_absolute_error': -0.004140817504411556}


In [38]:
predictions = predictor.predict(test_data.drop(columns=[label_column]))
print(predictions.head())

1775    0.508763
51      0.471811
194     0.814323
2756    0.036807
2019    0.144466
Name: fantasy_points_ppr_last_norm, dtype: float32


In [39]:
test_data[label_column]

1775    0.498282
51      0.478611
194     0.797413
2756    0.033418
2019    0.131998
          ...   
2250    0.104640
611     0.133128
1506    0.062585
3173    0.239396
1569    0.109614
Name: fantasy_points_ppr_last_norm, Length: 692, dtype: float64

### Reversing the normalization

In [40]:
def reverse_min_max_scaling(normalized_data, min_v, max_v):
    return normalized_data * (max_v - min_v) + min_v

In [42]:
min_v = df_to_norm['fantasy_points_ppr_last'].min()
max_v = df_to_norm['fantasy_points_ppr_last'].max()
original_predictions = reverse_min_max_scaling(test_data[label_column].copy(), min_v, max_v)

In [43]:
original_predictions

1775    217.6
51      208.9
194     349.9
2756     12.0
2019     55.6
        ...  
2250     43.5
611      56.1
1506     24.9
3173    103.1
1569     45.7
Name: fantasy_points_ppr_last_norm, Length: 692, dtype: float64

In [44]:
model_predictions = reverse_min_max_scaling(predictions.copy(), min_v, max_v)

In [45]:
model_predictions

1775    222.235580
51      205.892548
194     357.378967
2756     13.499070
2019     61.114342
           ...    
2250     39.957054
611      56.000145
1506     27.601654
3173    103.513145
1569     48.760220
Name: fantasy_points_ppr_last_norm, Length: 692, dtype: float32

In [49]:
close_enough = 0
far = 0
for guess, answer in zip(model_predictions, original_predictions):
    diff = abs(answer - guess)
    if diff <= 5:
        close_enough += 1
    elif diff >= 30:
        far += 1

In [50]:
close_enough

586

In [51]:
len(model_predictions)

692

In [52]:
close_enough / len(model_predictions)

0.846820809248555

In [53]:
far

0

## Further Information

In [54]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.009135,-0.010490,root_mean_squared_error,0.053105,0.030232,70.860849,0.001822,0.000262,0.008698,2,True,9
1,CatBoost,-0.010234,-0.011890,root_mean_squared_error,0.007746,0.003347,61.686298,0.007746,0.003347,61.686298,1,True,4
2,NeuralNetFastAI,-0.010745,-0.012030,root_mean_squared_error,0.016949,0.010821,2.128751,0.016949,0.010821,2.128751,1,True,6
3,XGBoost,-0.012907,-0.013668,root_mean_squared_error,0.015107,0.007608,1.276768,0.015107,0.007608,1.276768,1,True,7
4,RandomForestMSE,-0.013352,-0.015189,root_mean_squared_error,0.075987,0.041943,1.897825,0.075987,0.041943,1.897825,1,True,3
5,ExtraTreesMSE,-0.013516,-0.014503,root_mean_squared_error,0.066878,0.040304,0.614605,0.066878,0.040304,0.614605,1,True,5
6,NeuralNetTorch,-0.014639,-0.015197,root_mean_squared_error,0.011482,0.008194,5.760334,0.011482,0.008194,5.760334,1,True,8
7,KNeighborsDist,-0.030595,-0.031341,root_mean_squared_error,0.037185,0.011722,0.171804,0.037185,0.011722,0.171804,1,True,2
8,KNeighborsUnif,-0.031457,-0.032242,root_mean_squared_error,0.016689,0.065140,3.733471,0.016689,0.065140,3.733471,1,True,1


For feature clarification:
- **yptmpa:** receiving yards per team pass attempt

In [55]:
predictor.feature_importance(data=test_data)

These features in provided data are not utilized by the predictor and will be ignored: ['yac_sh_last_norm']
Computing feature importance via permutation shuffling for 42 features using 692 rows with 5 shuffle sets...
	13.59s	= Expected runtime (2.72s per shuffle set)
	2.86s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
rank_last_norm,0.057566,0.001330,3.421748e-08,5,0.060306,0.054827
receiving_yards_last_norm,0.043922,0.001039,3.760303e-08,5,0.046062,0.041781
receptions_last_norm,0.027363,0.000731,6.120259e-08,5,0.028869,0.025857
receiving_tds_last_norm,0.025367,0.000314,2.830561e-09,5,0.026014,0.024719
tier_last_norm,0.015651,0.000337,2.578619e-08,5,0.016345,0.014957
ppr_sh_last_norm,0.014513,0.000589,3.256897e-07,5,0.015726,0.013299
targets_last_norm,0.011438,0.000357,1.131176e-07,5,0.012172,0.010704
receiving_first_downs_last_norm,0.011098,0.000459,3.511071e-07,5,0.012044,0.010153
games_last_norm,0.005233,0.000242,5.428886e-07,5,0.005731,0.004736
yptmpa_last_norm,0.003984,0.000432,1.639723e-05,5,0.004875,0.003094
